# Modelling - exploration

## 1. Imports and Reading data

In [1]:
from runner import io 

import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as matplotlib
from scipy.stats import loguniform


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score,roc_auc_score, accuracy_score

from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [34]:
df = io.load_table("model", "model_df")
df

,year,nb_heads_cattle,area_ha_algodão_herbáceo_(em_caroço),area_ha_arroz_(em_casca),area_ha_café_(em_grão)_total,area_ha_cana-de-açúcar,area_ha_feijão_(em_grão),area_ha_laranja,area_ha_mandioca,area_ha_milho_(em_grão),...,quantidade_ton_cana-de-açúcar,quantidade_ton_feijão_(em_grão),quantidade_ton_laranja,quantidade_ton_mandioca,quantidade_ton_milho_(em_grão),quantidade_ton_soja_(em_grão),quantidade_ton_trigo_(em_grão),city,deforestation_ha,location_UF
0,1986,513300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4200.0,56.0,11280.0,6300.0,1512.0,0.0,0.0,Aquidauana,0.000000,MS
1,1986,93300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1200.0,645.0,5925.0,1500.0,3600.0,135.0,0.0,Bodoquena,0.000000,MS
2,1986,1418523.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,600.0,316.0,NaN,2850.0,2160.0,0.0,4.0,Corumbá,4784.695101,MS
3,1986,387006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,200.0,150.0,180.0,7000.0,10500.0,27000.0,0.0,Coxim,0.000000,MS
4,1986,12361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.0,48.0,0.0,300.0,270.0,0.0,0.0,Ladário,637.707553,MS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,2020,336622.0,6131.0,0.0,0.0,5000.0,200.0,0.0,4.0,92000.0,...,360000.0,240.0,0.0,40.0,593400.0,604800.0,0.0,Itiquira,4330.575567,MT
546,2020,161266.0,0.0,0.0,0.0,9452.0,500.0,0.0,63.0,340.0,...,829508.0,500.0,0.0,791.0,1944.0,3647.0,0.0,Lambari D'Oeste,0.000000,MT
547,2020,185903.0,0.0,8.0,0.0,200.0,0.0,0.0,160.0,600.0,...,6100.0,0.0,0.0,1150.0,3450.0,2016.0,0.0,Nossa Senhora do Livramento,5735.257631,MT
548,2020,531977.0,0.0,20.0,0.0,200.0,20.0,0.0,140.0,660.0,...,11000.0,12.0,0.0,1400.0,2785.0,32068.0,0.0,Poconé,9803.755779,MT


## 2. Modelling

In [10]:
X = df.drop(columns=["deforestation_ha"])
y = df["deforestation_ha"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
cat_vars_complete = [i for i in X_train .columns if X_train.dtypes[i] == 'object']
num_vars_complete = [i for i in X_train.columns if X_train.dtypes[i] == 'float64' or X_train.dtypes[i] == 'int64']

ignore_variables = []
cat_vars = [element for element in cat_vars_complete if (element not in ignore_variables)]
specific_vars = []
num_vars = [element for element in num_vars_complete if (element != 'deforestation_ha' and element not in specific_vars + ignore_variables)]


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
#     ('polynomial_features',PolynomialFeatures(interaction_only=True)),
#     ('pca', PCA())
])

specific_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('power_transform', PowerTransformer(method='yeo-johnson')),
#     ('polynomial_features',PolynomialFeatures(interaction_only=False)),
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
#     ('truncated_svd', TruncatedSVD())
#     ('sparse_pca', sparse_pca),
#     ('to_dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)
    ])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_vars),
        ('cat', categorical_transformer, cat_vars),
#         ('specific', specific_transformer, specific_vars),
    ])

rf_regressor = RandomForestRegressor()

# param_dist_rf = {"max_depth": [None, 1],
#               "max_features": ['auto','sqrt'],
#               "min_samples_split": [2,3],
#               "min_samples_leaf": [3, 4],
#               "n_estimators": [400]}

# rf_random_search = RandomizedSearchCV(rf_regressor, param_distributions=param_dist_rf)

model_rf = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', rf_regressor)
        ]
    )


In [29]:
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

In [30]:
from sklearn.metrics import r2_score, mean_squared_error
r2_score(y_test, y_pred)

0.4093461231493699

In [36]:
mean_squared_error(y_test, y_pred, squared=False)/np.mean(y_test)

0.9839157939397575